# 🍽️ Task-05: Food Recognition and Calorie Estimation
This notebook builds a CNN model to classify food images from the [Food-101 dataset](https://www.kaggle.com/dansbecker/food-101) and estimates calorie values for each class to help track dietary intake.

## 📥 1. Setup & Kaggle Dataset Download

In [ ]:
# Install Kaggle and set up API key (Upload kaggle.json manually before running this)
!pip install -q kaggle

# Upload kaggle.json (from your Kaggle account)
from google.colab import files
files.upload()

# Move kaggle.json to correct path
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Food-101 dataset
!kaggle datasets download -d dansbecker/food-101
!unzip -q food-101.zip


## 🧹 2. Preprocess the Dataset

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

# Paths
data_dir = 'food-101/images'

# Image Data Generator for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


## 🧠 3. Build and Train CNN Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)


## 📈 4. Evaluate Model & Plot Accuracy

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()


## 🧮 5. Calorie Estimation (Example Mapping)

In [ ]:
# Simple example: map food labels to estimated calories
food_calories = {
    'apple_pie': 300,
    'pizza': 285,
    'sushi': 200,
    'ice_cream': 210,
    # Add more manually...
}

# Example usage
predicted_class = 'apple_pie'
print(f"Predicted food: {predicted_class} | Estimated calories: {food_calories.get(predicted_class, 'N/A')} kcal")
